
# Kapitel 4 – Reguläre Ausdrücke zur Textanalyse (Begleitnotebook)

**Hinweis:** Dieses Notebook nutzt **andere Beispiele** als das Skript.  
Alle Texte sind **synthetisch** (historisch inspiriert). **Keine externen Daten** werden geladen.

## Lernziele
- RegEx-Grundbausteine (Zeichenklassen, Quantoren, Anker, Gruppen, Alternativen)
- Python-Workflow mit `re`: `findall`, `search`, `finditer`, `sub`, `split`, `compile`
- Greedy vs. non-greedy
- Lookahead/Lookbehind
- Gruppen (positional & named) und Backreferences
- Flags: `IGNORECASE`, `MULTILINE`, `DOTALL`, `VERBOSE`

## Setup

In [1]:
import re

def headline(txt):
    print("\n" + "="*len(txt))
    print(txt)
    print("="*len(txt))

# Synthetic, historically flavored snippets (NOT from the script)
latin_snippet = (
    "Anno DCCLIII ab urbe condita legati Romani ad Helvetios missi sunt.\n"
    "M. Tullius Cicero orationem habuit in foro Romano die VIII Kal. Mart.\n"
    "In tabulis publicis legitur: 'civibus libertatem confirmamus'."
)

german_snippet = (
    "Im Jahre 1525 schrieb ein Chronist in Nürnberg: \n"
    "Joh. Stumpf berichtet am 12.03.1525 über den Umzug. \n"
    "In der Stadtchronik steht: »Wir bestellten die Wächter; die Tore blieben zu.«\n"
    "Siegel-Nr. A-017 wurde an Tor B-3 angebracht."
)

print("Beispiele geladen (synthetisch).")

Beispiele geladen (synthetisch).



## 1) Mini-Cheatsheet (Python `re`)

- **Punkt** `.`: ein beliebiges Zeichen (außer Zeilenumbruch, außer mit `DOTALL`)
- **Quantoren**: `*` (0+), `+` (1+), `?` (0/1), `{m}`, `{m,}`, `{m,n}`
- **Zeichenklassen**: `[abc]`, `[A-Z]`, `\d` (Ziffer), `\w` (alphanum + `_`), `\s` (Whitespace) und Negationen `\D`, `\W`, `\S`
- **Anker**: `^` (Anfang), `$` (Ende), `\b` (Wortgrenze)
- **Gruppen**: `( ... )`, **Alternativen**: `A|B`
- **Lookarounds**: `(?=...)` (Lookahead), `(?!...)` (neg. Lookahead), `(?<=...)` (Lookbehind), `(?<!...)` (neg. Lookbehind)


## 2) Grundlegende Suche mit `findall`

In [2]:
headline("Wörter, die mit Großbuchstaben beginnen (Latein)")
print(re.findall(r"\b[A-Z][a-z]+\b", latin_snippet))

headline("Alle römisch anmutenden Abkürzungen wie 'M.' oder 'Kal.'")
print(re.findall(r"\b[A-Z][a-z]*\.", latin_snippet))

headline("Deutsche Datumsangaben im Format DD.MM.YYYY")
print(re.findall(r"\b\d{1,2}\.\d{1,2}\.\d{4}\b", german_snippet))

headline("Signaturen wie 'A-017' oder 'B-3'")
print(re.findall(r"\b[A-Z]-\d{1,3}\b", german_snippet))


Wörter, die mit Großbuchstaben beginnen (Latein)
['Anno', 'Romani', 'Helvetios', 'Tullius', 'Cicero', 'Romano', 'Kal', 'Mart', 'In']

Alle römisch anmutenden Abkürzungen wie 'M.' oder 'Kal.'
['M.', 'Kal.', 'Mart.']

Deutsche Datumsangaben im Format DD.MM.YYYY
['12.03.1525']

Signaturen wie 'A-017' oder 'B-3'
['A-017', 'B-3']


## 3) `search` & Spannen (`span`)

In [3]:
m = re.search(r"\bCicero\b", latin_snippet)
print(m)
if m:
    print("Span:", m.span())
    start, end = m.span()
    print("Gefundenes Stück:", latin_snippet[start:end])

<re.Match object; span=(79, 85), match='Cicero'>
Span: (79, 85)
Gefundenes Stück: Cicero


## 4) `re.compile` – einmal definieren, oft benutzen

In [4]:
name_pat = re.compile(r"\b[A-Z][a-z]+\b")
print(name_pat.findall(latin_snippet))

['Anno', 'Romani', 'Helvetios', 'Tullius', 'Cicero', 'Romano', 'Kal', 'Mart', 'In']


## 5) `finditer` für strukturierte Extraktion

In [6]:
pattern = re.compile(r"\b([A-Z][a-z]*\.)\s+([A-Z][a-z]+)\b")  # z.B. 'M. Tullius'
for match in pattern.finditer(latin_snippet):
    abbr, cognomen = match.groups()
    print({"Abkürzung": abbr, "Name": cognomen, "span": match.span()})

{'Abkürzung': 'M.', 'Name': 'Tullius', 'span': (68, 78)}
{'Abkürzung': 'Kal.', 'Name': 'Mart', 'span': (127, 136)}


## 6) Gruppen & Named Groups

In [7]:
date_pattern = re.compile(r"(?P<tag>\d{2})\.(?P<monat>\d{2})\.(?P<jahr>\d{4})")
for m in date_pattern.finditer(german_snippet):
    print("Tag:", m.group("tag"), "Monat:", m.group("monat"), "Jahr:", m.group("jahr"))

Tag: 12 Monat: 03 Jahr: 1525


## 7) Greedy vs. Non-greedy

In [8]:
quote_text = "In der Chronik steht: »Wir bestellten die Wächter; die Tore blieben zu.« Weiter heißt es: »Dies sei bekannt.«"
headline("Greedy (.*)")
print(re.findall(r"».*«", quote_text))

headline("Non-greedy (.*?)")
print(re.findall(r"».*?«", quote_text))


Greedy (.*)
['»Wir bestellten die Wächter; die Tore blieben zu.« Weiter heißt es: »Dies sei bekannt.«']

Non-greedy (.*?)
['»Wir bestellten die Wächter; die Tore blieben zu.«', '»Dies sei bekannt.«']


## 8) Lookahead & Lookbehind

In [9]:
headline("Lookahead: Wörter 'in' und 'In', denen ein Wort mit 'd' folgt (Deutsch)")
print(re.findall(r"\s[Ii]n(?=\s[dD])", german_snippet))

headline("Lookbehind: 'in' unmittelbar nach 't' (Latein)")
print(re.findall(r"(?<=t\s)in\b", latin_snippet))

headline("Kombiniert: Wort vor 'in' endet auf 't', danach folgt 'f'")
print(re.findall(r"(?<=t\s)in(?=\sf)", latin_snippet))


Lookahead: Wörter 'in' und 'In', denen ein Wort mit 'd' folgt (Deutsch)
['\nIn']

Lookbehind: 'in' unmittelbar nach 't' (Latein)
['in']

Kombiniert: Wort vor 'in' endet auf 't', danach folgt 'f'
['in']


## 9) Alternativen & Backreferences

In [11]:
headline("Alternativen: Monatskürzel (Kal.|Id.|Non.) extrahieren")
print(re.findall(r"\b(Kal\.|Id\.|Non\.)\b", latin_snippet))

headline("Backreference: doppelte Anführungen »...« oder „...“")
mixed = "»Salvete!« sagte er. „Valete!“ antwortete sie. »Iterum?«"
print(re.findall(r"(»|„|«)(.*?)\1", mixed))  # Gruppe 1 wird am Ende wiederverwendet


Alternativen: Monatskürzel (Kal.|Id.|Non.) extrahieren
[]

Backreference: doppelte Anführungen »...« oder „...“
[('»', 'Salvete!« sagte er. „Valete!“ antwortete sie. ')]


## 10) `re.sub`: Cleanup-Beispiele

In [12]:
# Beispiel 1: Bindestrich-Trennungen über Zeilen umbrechen (OCR)
ocr = "trans-\nmarinus nego-\ntium per-\nactum"
fixed = re.sub(r"-\n", "", ocr)   # weiches Silbentrennzeichen entfernen
print("Vorher:", ocr)
print("Nachher:", fixed)

# Beispiel 2: Editionszeichen entfernen: eckige Klammern & Klammerauflösungen
edition = "C(aius) Iul[i]us prae[f]ectus"
clean = re.sub(r"[\[\])(]", "", edition)
print("Vorher:", edition)
print("Nachher:", clean)

Vorher: trans-
marinus nego-
tium per-
actum
Nachher: transmarinus negotium peractum
Vorher: C(aius) Iul[i]us prae[f]ectus
Nachher: Caius Iulius praefectus


## 11) `re.split`, Flags (`IGNORECASE`, `MULTILINE`, `DOTALL`, `VERBOSE`)

In [15]:
# Split an Satzendzeichen (einfach, nicht linguistisch vollständig)
text = "Roma vicit. sed pax servanda est! quid tum?"
print(re.split(r"[.!?]\s*", text))

# Flags: IGNORECASE & MULTILINE
multi = "caput I\nCaput II\ncaput III"
print(re.findall(r"^caput\s+[IVX]+$", multi, flags=re.IGNORECASE|re.MULTILINE))

# DOTALL: Punkt matcht Zeilenumbrüche
block = "Titulus: Res Gestae\n---\nDivi Augusti"
print(re.findall(r"Titulus:.*Divi", block, flags=re.DOTALL))

# VERBOSE: Lesbares Pattern (mit Kommentaren)
pattern = re.compile(r"""
    (?P<id>[A-Z]-\d{1,3})     # Signatur wie A-017
    \s+an\s+Tor\s+
    (?P<tor>[A-Z]-\d)         # Torkennung wie B-3
""", re.VERBOSE)
for m in pattern.finditer(german_snippet):
    print(m.groupdict())

['Roma vicit', 'sed pax servanda est', 'quid tum', '']
['caput I', 'Caput II', 'caput III']
['Titulus: Res Gestae\n---\nDivi']



## 12) Mini-Aufgaben

**A1.** Extrahiere aus `latin_snippet` alle Datumsangaben römischer Art **(nur)**: `\b[IVX]+\b` (z. B. `VIII`).  
Wandle die Treffer in eine Liste eindeutiger Werte um.

**A2.** Finde im `german_snippet` alle **Anführungssegmente** in »…« **und** „…“. Gib den **Inhalt ohne Anführungszeichen** aus.

**A3.** Ersetze im `german_snippet` alle **Bindestrich-Signaturen** `A-017`, `B-3` durch die Form `A.017`, `B.3` (mittels `re.sub`).

**A4.** Nutze einen **Lookbehind**, um alle Vorkommen von `in` zu finden, denen **sofort** ein Wort folgt, das mit **Buchstaben** beginnt (kein Digit).

---
**Lösungsskizzen** (Zellen unten ausführen, aber versuche es zuerst selbst):


In [ ]:
# Lösungsskizze A1
roman_dates = set(re.findall(r"\b[IVX]+\b", latin_snippet))
print(sorted(roman_dates))

In [ ]:
# Lösungsskizze A2
quotes = re.findall(r"(»|„)(.*?)\1", german_snippet)
contents = [c for _, c in quotes]
print(contents)

In [ ]:
# Lösungsskizze A3
print(re.sub(r"\b([A-Z])-(\d{1,3})\b", r"\1.\2", german_snippet))

In [ ]:
# Lösungsskizze A4
print(re.findall(r"\bin(?=\s[\p{L}A-Za-z])", german_snippet))  # Fallback: A-Za-z falls \p{L} nicht unterstützt
# Robustere Variante ohne \p{L}:
print(re.findall(r"\bin(?=\s[A-Za-zÄÖÜäöüß])", german_snippet))


## 13) Weiterführend
- Python-Doku `re`: https://docs.python.org/3/library/re.html  
- Online-Tester: https://regex101.com (hilfreich zum Debuggen)
- Faustregeln:
  - Greedy → oft zu weitreichend → mit `?` **non-greedy** machen
  - **Lookarounds** für kontextsensitives Matching, ohne Kontext mitzumatchen
  - Bei Editionstexten: **erst reinigen**, dann extrahieren
  - Komplexe Muster → **`re.VERBOSE`** nutzen (lesbar + kommentierbar)
